In [16]:
%%capture --no-stderr
!pip install langchain-ollama
!pip install langchain-nomic "nomic[local]"
!pip install langchain
!pip install tiktoken scikit-learn
!pip install --quiet -U langchain langchain_community tiktoken langchain-nomic "nomic[local]" langchain-ollama scikit-learn langgraph tavily-python bs4

In [32]:
from langchain_ollama import ChatOllama
model_id = "llama3.2:1b"
llm = ChatOllama(model=model_id, temperature=0)

### LLM


In [48]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_nomic.embeddings import NomicEmbeddings
from langchain_ollama import ChatOllama

urls = [
    "https://www.wcpss.net/site/default.aspx?PageType=3&DomainID=15369&ModuleInstanceID=82538&ViewID=6446EE88-D30C-497E-9316-3F8874B3E108&RenderLoc=0&FlexDataID=326169&PageID=39173",
    "https://www.wcpss.net/site/default.aspx?PageType=3&DomainID=15369&ModuleInstanceID=82538&ViewID=6446EE88-D30C-497E-9316-3F8874B3E108&RenderLoc=0&FlexDataID=325688&PageID=39173",
    "https://www.wcpss.net/site/default.aspx?PageType=3&DomainID=15369&ModuleInstanceID=82538&ViewID=6446EE88-D30C-497E-9316-3F8874B3E108&RenderLoc=0&FlexDataID=325422&PageID=39173",
    "https://www.wcpss.net/site/default.aspx?PageType=3&DomainID=15369&ModuleInstanceID=82538&ViewID=6446EE88-D30C-497E-9316-3F8874B3E108&RenderLoc=0&FlexDataID=325129&PageID=39173"
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=200
)
doc_splits = text_splitter.split_documents(docs_list)

vectorstore = SKLearnVectorStore.from_documents(
    documents=doc_splits,
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
)

retriever = vectorstore.as_retriever(k=3)



Embedding texts: 100%|██████████| 30/30 [00:43<00:00,  1.44s/inputs]


In [49]:
print(model)

model='llama3.2:1b' temperature=0.0


In [ ]:
model_id = "llama3.2:1b"
model = ChatOllama(model=model_id, temperature=0)

query = "What is LeBron James 2025 stats?"

retrieved_docs = retriever.get_relevant_documents(query)

prompt = f"Based on the following information, answer the question: {query}\n\n" + "\n".join([doc.page_content for doc in retrieved_docs])

response = model.generate(prompt)
print(response)

In [34]:
### Router
import json
from langchain_core.messages import HumanMessage, SystemMessage

# Prompt
router_instructions = """You are an expert at answering questions from a vectorstore.

Use the vectorstore for questions. """

# Test router
test_web_search = llm.invoke(
    [SystemMessage(content=router_instructions)]
    + [
        HumanMessage(
            content="What is Lebron James 2025 stats?"
        )
    ]
)

print(
    test_web_search
)

content="I'm not aware of any information about Lebron James' 2025 stats, as my knowledge cutoff is March 1, 2023. However, I can suggest some possible sources where you may be able to find the information you're looking for:\n\n* Official NBA website: You can check out LeBron James' official NBA profile on the league's website, which should have his career statistics and other relevant information.\n* Sports websites: Websites like ESPN, Fox Sports, or CBS Sports often provide up-to-date stats and analysis of NBA players, including Lebron James.\n* Social media: You can also try checking out LeBron James' social media accounts to see if he has shared any updates about his 2025 stats.\n\nIf you're looking for specific stats from a particular season or game, I'd be happy to try and help you find that information." additional_kwargs={} response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-02-16T15:40:21.4582317Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7063892200

In [58]:
### Generate
question = "When does the book club meet"
# Prompt
rag_prompt = """You are an assistant for question-answering tasks about Green Level High School. 

Here is the context to use to answer the question:

{context} 

Think carefully about the above context. 

Now, review the user question:

{question}

Provide an answer to this questions using only the above context. 

Use three sentences maximum and keep the answer concise.

Answer:"""


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Test
docs = retriever.invoke(question)
docs_txt = format_docs(docs)
rag_prompt_formatted = rag_prompt.format(context=docs_txt, question=question)
generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
print(generation.content)

Embedding texts: 100%|██████████| 1/1 [00:00<00:00, 28.77inputs/s]


The Green Level Book Club meets on Tuesdays during Connectivity (register with Ms. Beach) from 1:00-2:00 pm, and also virtually at 6:00-7:00 pm on Fridays.


In [ ]:
 prompt = "What is Lebron James 2025 stats"
 response = model.invoke(prompt)
 print(response)